In [1]:
import logging


# ---------------------------
# Logging Configuration
# ---------------------------
def setup_logger():
    logger = logging.getLogger("my_app")
    logger.setLevel(logging.DEBUG)  # capture everything

    # Formatter
    formatter = logging.Formatter(
        "%(asctime)s | %(name)s | %(levelname)s | %(message)s"
    )

    # Console handler
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    console_handler.setFormatter(formatter)

    # File handler
    file_handler = logging.FileHandler("app.log")
    file_handler.setLevel(logging.DEBUG)
    file_handler.setFormatter(formatter)

    # Avoid duplicate handlers
    if not logger.handlers:
        logger.addHandler(console_handler)
        logger.addHandler(file_handler)

    return logger


logger = setup_logger()


# ---------------------------
# Functions with logging
# ---------------------------
def divide(a, b):
    logger.debug(f"divide() called with a={a}, b={b}")
    try:
        result = a / b
        logger.info(f"Division successful: {result}")
        return result
    except ZeroDivisionError:
        logger.error("Attempted division by zero", exc_info=True)
        return None


# ---------------------------
# Class with logging
# ---------------------------
class BankAccount:
    def __init__(self, owner, balance=0):
        self.owner = owner
        self.balance = balance
        logger.info(f"BankAccount created for {owner} with balance {balance}")

    def deposit(self, amount):
        logger.debug(f"Deposit called with amount={amount}")
        if amount <= 0:
            logger.warning("Deposit amount must be positive")
            return

        self.balance += amount
        logger.info(f"Deposited {amount}. New balance: {self.balance}")

    def withdraw(self, amount):
        logger.debug(f"Withdraw called with amount={amount}")
        if amount > self.balance:
            logger.error(
                f"Insufficient funds: balance={self.balance}, requested={amount}"
            )
            return

        self.balance -= amount
        logger.info(f"Withdrawn {amount}. New balance: {self.balance}")


# ---------------------------
# Main execution
# ---------------------------
if __name__ == "__main__":
    logger.info("Application started")

    divide(10, 2)
    divide(10, 0)

    account = BankAccount("Gopal", 1000)
    account.deposit(500)
    account.withdraw(2000)
    account.withdraw(300)

    logger.critical("Application finished with critical example")

2026-02-05 13:37:03,415 | my_app | INFO | Application started
2026-02-05 13:37:03,421 | my_app | INFO | Division successful: 5.0
2026-02-05 13:37:03,423 | my_app | ERROR | Attempted division by zero
Traceback (most recent call last):
  File "C:\Users\GOPALAKRISHNANSUBRAM\AppData\Local\Temp\ipykernel_11528\73015563.py", line 43, in divide
    result = a / b
ZeroDivisionError: division by zero
2026-02-05 13:37:03,425 | my_app | INFO | BankAccount created for Gopal with balance 1000
2026-02-05 13:37:03,426 | my_app | INFO | Deposited 500. New balance: 1500
2026-02-05 13:37:03,427 | my_app | ERROR | Insufficient funds: balance=1500, requested=2000
2026-02-05 13:37:03,428 | my_app | INFO | Withdrawn 300. New balance: 1200
2026-02-05 13:37:03,429 | my_app | CRITICAL | Application finished with critical example


In [5]:
# with decorator, instrument

import logging
import time
import functools

logger = logging.getLogger("instrument")


def instrument(name: str | None = None):
    """
    Decorator to log BEGIN / END with execution time.
    Can be used on functions or methods.
    """

    def decorator(func):
        display_name = name or func.__qualname__

        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            logger.info(f"BEGIN {display_name}")
            start = time.perf_counter()

            try:
                result = func(*args, **kwargs)
                return result
            except Exception:
                logger.exception(f"EXCEPTION in {display_name}")
                raise
            finally:
                duration = time.perf_counter() - start
                logger.info(
                    f"END {display_name} | time={duration:.6f}s"
                )

        return wrapper

    return decorator


In [3]:

import logging

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s"
)



In [7]:
import time


@instrument()
def slow_add(a, b):
    time.sleep(0.5)
    return a + b


slow_add(2, 3)

2026-02-05 13:40:43,101 | INFO | BEGIN slow_add
2026-02-05 13:40:43,620 | INFO | END slow_add | time=0.513020s


5